# EDA

В README не объясняют, что за задачу мы решаем и какие у нас данные, надо разобраться.


* В файле `problem_train.csv` в каждой строке содержится информация об объекте
с уникальным идентификатором, заданным в столбце `id`.

* В файле `problem_labels.csv` для каждого `id` из файла `problem_train.csv` приведена информация о
принадлежности к 14 категориям.

* В файле `problem_test.csv` дана информация об объектах, аналогичная содержащейся в файле
`problem_train.csv`.

* Используя `problem_train.csv` и `problem_labels.csv` в качестве данных для обучения, постройте файл
`problem_test_labels.csv`, в котором для каждого объекта из файла `problem_test.csv` укажите
вероятность его принадлежности к каждой из 14 рассматриваемых категорий.

* Для оценки качества полученного результата будет использоваться метрика `LogLoss`. По каждой из 14 
категорий метрика считается независимо, затем берется среднее арифметическое значений метрик,
полученных для категорий.

* Пожалуйста, предоставьте краткое описание пути поиска решения и код, использованный для получения
результата.

In [1]:
from IPython.core.display import HTML

HTML(
    r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: "JetBrainsMono Nerd Font Mono"; # Any monospaced font should work
    }
</style>
"""
)

In [2]:
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import pandas as pd

In [3]:
pio.templates.default = "plotly_dark"

## TODO: change to white before submitting

### Откроем данные

In [4]:
train_df = pd.read_csv("./data/problem_train.csv", low_memory=False)

In [5]:
train_df

,id,release,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,...,c_1368,c_1369,c_1370,c_1371,c_1372,c_1373,c_1374,c_1375,c_1376,c_1377
0,11193,a,NaN,NaN,0.025449,NaN,NaN,0.368421,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,q,NaN,NaN,NaN
1,11382,a,NaN,NaN,0.031297,NaN,NaN,0.315789,NaN,NaN,...,NaN,NaN,a,NaN,a,NaN,NaN,NaN,NaN,NaN
2,16531,a,NaN,NaN,0.024475,NaN,NaN,0.342105,NaN,NaN,...,NaN,NaN,a,NaN,a,NaN,b,NaN,NaN,NaN
3,1896,a,NaN,NaN,0.041694,NaN,NaN,0.447368,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN
4,18262,c,NaN,NaN,0.038120,NaN,NaN,0.315789,NaN,NaN,...,NaN,NaN,b,NaN,a,NaN,a,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,10898,a,NaN,NaN,0.053931,NaN,NaN,0.394737,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,b
7996,16664,a,NaN,NaN,0.031731,NaN,NaN,0.394737,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,a,NaN,NaN,NaN
7997,5334,c,NaN,0.904762,0.033463,NaN,NaN,0.394737,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN
7998,7905,c,NaN,NaN,0.047109,NaN,NaN,0.289474,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,q,NaN,NaN,NaN


In [6]:
desc = train_df.describe(include="all")
desc

,id,release,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,...,c_1368,c_1369,c_1370,c_1371,c_1372,c_1373,c_1374,c_1375,c_1376,c_1377
count,8000.000000,8000,12.000000,388.000000,7662.000000,112.000000,58.000000,7658.000000,1348.000000,995.000000,...,3,48,377,2,7998,440,3155,563,7,1908
unique,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,11,3,2,2,5,24,19,2,2
top,NaN,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,d,m,b,c,a,c,b,j,b,b
freq,NaN,3838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,22,312,1,7460,167,1559,232,4,1441
mean,9220.303375,NaN,0.307692,0.655744,0.040935,0.102749,0.352874,0.395981,0.193175,0.012012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,5236.897900,NaN,0.245454,0.216399,0.024599,0.153007,0.209337,0.085000,0.236683,0.041382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,4773.500000,NaN,0.134615,0.428571,0.031974,0.011905,0.200000,0.342105,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,9221.500000,NaN,0.307692,0.702381,0.041369,0.047619,0.283333,0.394737,0.200000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,13702.750000,NaN,0.403846,0.857143,0.048950,0.142857,0.525000,0.421053,0.400000,0.011905,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Удостоверимся, что id не повторяется

In [7]:
train_df.id.duplicated().any()

False

#### Как определены фичи

In [8]:
px.histogram(desc.loc["count"])

In [9]:
na_cols = desc.loc["count"][desc.loc["count"] == 0].index
na_cols

Index(['n_0101', 'o_0224', 'o_0257', 'o_0308', 'c_0491', 'c_0529', 'c_0541',
       'c_0596', 'c_0632', 'c_0644', 'c_0693', 'c_0708', 'c_0710', 'c_0763',
       'c_0784', 'c_0803', 'c_0848', 'c_0902', 'c_0959', 'c_1009', 'c_1072',
       'c_1129', 'c_1148', 'c_1168', 'c_1230', 'c_1257', 'c_1277', 'c_1281',
       'c_1308', 'c_1332'],
      dtype='object')

In [10]:
train_df

,id,release,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,...,c_1368,c_1369,c_1370,c_1371,c_1372,c_1373,c_1374,c_1375,c_1376,c_1377
0,11193,a,NaN,NaN,0.025449,NaN,NaN,0.368421,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,q,NaN,NaN,NaN
1,11382,a,NaN,NaN,0.031297,NaN,NaN,0.315789,NaN,NaN,...,NaN,NaN,a,NaN,a,NaN,NaN,NaN,NaN,NaN
2,16531,a,NaN,NaN,0.024475,NaN,NaN,0.342105,NaN,NaN,...,NaN,NaN,a,NaN,a,NaN,b,NaN,NaN,NaN
3,1896,a,NaN,NaN,0.041694,NaN,NaN,0.447368,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN
4,18262,c,NaN,NaN,0.038120,NaN,NaN,0.315789,NaN,NaN,...,NaN,NaN,b,NaN,a,NaN,a,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,10898,a,NaN,NaN,0.053931,NaN,NaN,0.394737,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,b
7996,16664,a,NaN,NaN,0.031731,NaN,NaN,0.394737,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,a,NaN,NaN,NaN
7997,5334,c,NaN,0.904762,0.033463,NaN,NaN,0.394737,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN
7998,7905,c,NaN,NaN,0.047109,NaN,NaN,0.289474,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,q,NaN,NaN,NaN


### Посмотрим, какие размерности у категориальных фичей

In [11]:
px.histogram(desc.loc["unique"])

### Посмотрим на типы данных

In [12]:
train_df.dtypes

id           int64
release     object
n_0000     float64
n_0001     float64
n_0002     float64
            ...   
c_1373      object
c_1374      object
c_1375      object
c_1376      object
c_1377      object
Length: 1379, dtype: object

In [13]:
train_df.dtypes.value_counts()

object     1025
float64     345
int64         9
dtype: int64

In [14]:
train_df.columns

Index(['id', 'release', 'n_0000', 'n_0001', 'n_0002', 'n_0003', 'n_0004',
       'n_0005', 'n_0006', 'n_0007',
       ...
       'c_1368', 'c_1369', 'c_1370', 'c_1371', 'c_1372', 'c_1373', 'c_1374',
       'c_1375', 'c_1376', 'c_1377'],
      dtype='object', length=1379)

In [15]:
train_df.columns.to_series().apply(lambda name: name[:2]).value_counts()

c_    1050
o_     211
n_     116
id       1
re       1
dtype: int64

#### Есть колонки начинающиеся с `o_`, это что?

In [16]:
train_df.columns[
    train_df.columns.to_series().apply(lambda name: str.startswith(name, "o_"))
]

Index(['o_0116', 'o_0117', 'o_0118', 'o_0119', 'o_0120', 'o_0121', 'o_0122',
       'o_0123', 'o_0124', 'o_0125',
       ...
       'o_0317', 'o_0318', 'o_0319', 'o_0320', 'o_0321', 'o_0322', 'o_0323',
       'o_0324', 'o_0325', 'o_0326'],
      dtype='object', length=211)

In [17]:
train_df["o_0326"]

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
7995    11.0
7996     NaN
7997     NaN
7998     9.0
7999     NaN
Name: o_0326, Length: 8000, dtype: float64

#### Интересно, int64 -- не только id. Возможно, это те же n_, которые просто определились как int?

In [18]:
int_columns = train_df.dtypes[
    train_df.dtypes.apply(lambda dtype: dtype == np.int64)
].index
int_columns

Index(['id', 'n_0047', 'n_0050', 'n_0052', 'n_0061', 'n_0075', 'n_0091',
       'o_0176', 'o_0264'],
      dtype='object')

In [19]:
train_df[int_columns]

,id,n_0047,n_0050,n_0052,n_0061,n_0075,n_0091,o_0176,o_0264
0,11193,1,1,1,1,1,1,303,7
1,11382,1,1,1,1,1,1,293,7
2,16531,1,1,1,1,1,1,131,0
3,1896,1,1,1,1,1,1,113,4
4,18262,1,1,1,1,1,1,240,6
...,...,...,...,...,...,...,...,...,...
7995,10898,1,1,1,1,1,1,4,5
7996,16664,1,1,1,1,1,1,207,5
7997,5334,1,1,1,1,1,1,369,8
7998,7905,1,1,1,1,1,1,69,4


In [20]:
train_df[["n_0047", "n_0050", "n_0052", "n_0061", "n_0075", "n_0091"]].value_counts()

n_0047  n_0050  n_0052  n_0061  n_0075  n_0091
1       1       1       1       1       1         8000
dtype: int64

Какие-то бесполезные колонка, все значения 1
## TODO: удалить эти колонки???

### Подгрузим лейблы

In [21]:
train_labels_df = pd.read_csv("./data/problem_labels.csv")
train_labels_df

,id,service_a,service_b,service_c,service_d,service_e,service_f,service_g,service_h,service_i,service_j,service_k,service_l,service_m,service_n
0,11193,1,1,0,0,0,0,0,0,0,1,1,0,0,0
1,11382,0,0,0,0,0,0,0,0,0,1,1,0,0,0
2,16531,0,0,0,0,0,0,0,0,0,1,1,0,0,0
3,1896,0,0,0,1,0,0,0,0,0,1,0,1,0,0
4,18262,0,0,0,1,1,0,0,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,10898,1,1,1,0,0,0,1,0,0,1,1,1,1,1
7996,16664,0,0,0,0,0,0,0,0,0,1,1,0,0,0
7997,5334,1,0,0,0,0,0,0,0,0,1,1,0,0,0
7998,7905,1,1,1,0,0,0,0,0,0,1,1,0,0,0


Проверим, что айдишники совпадают

In [22]:
(train_labels_df.id == train_df.id).all()

True

Пока что ничего не понимаю. есть категориальные, флоут переменные. Возможно, это что-то нагенерили из исходных данных.

В названиях есть release, service -- мб это что-то связанное с разными продуктами? или с разными сервисами внутри тинькофф?

Одной записи соответствует несколько лейблов, то есть у нас мультилейбл задача

#### мб стоит взглянуть на матрицу корреляции между лейблами?)))

In [23]:
px.imshow(
    train_labels_df.drop(columns=["id"]).corr(),
)

Соседние лейблы в целом сильнее скоррелированы. например i-m = 0.80, j-k = 0.53, f-g = 0.40, d-e = 0.42, a-b = 0.56, b-c = 0.4.

Есть еще h-i = 0.30, c-g = 0.27

Эта корреляция мб не будет мешать модели, если на каждый будет отдельная модель

#### Все лейблы определены на всех записях

In [24]:
train_labels_df.describe()

,id,service_a,service_b,service_c,service_d,service_e,service_f,service_g,service_h,service_i,service_j,service_k,service_l,service_m,service_n
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.0000,8000.000000,8000.000000
mean,9220.303375,0.472125,0.328250,0.261625,0.017000,0.053875,0.029125,0.051875,0.297000,0.016125,0.848250,0.782000,0.1080,0.088625,0.178000
std,5236.897900,0.499254,0.469606,0.439547,0.129279,0.225785,0.168167,0.221788,0.456965,0.125964,0.358801,0.412913,0.3104,0.284219,0.382537
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
25%,4773.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.0000,0.000000,0.000000
50%,9221.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.0000,0.000000,0.000000
75%,13702.750000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.0000,0.000000,0.000000
max,18302.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0000,1.000000,1.000000


Вот только вопрос - 8000 это много или мало 🤔 Думаю, достаточно

### Посмотрим на тест

In [25]:
test_df = pd.read_csv("./data/problem_test.csv", low_memory=False)
test_df

,id,release,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,...,c_1368,c_1369,c_1370,c_1371,c_1372,c_1373,c_1374,c_1375,c_1376,c_1377
0,7957,a,NaN,NaN,0.029890,NaN,NaN,0.368421,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,q,NaN,NaN,NaN
1,693,c,NaN,0.452381,0.032164,NaN,NaN,0.421053,0.0,NaN,...,NaN,NaN,NaN,NaN,a,NaN,x,NaN,NaN,NaN
2,15558,c,NaN,0.547619,0.049166,NaN,NaN,0.500000,0.2,NaN,...,NaN,NaN,NaN,NaN,a,e,b,NaN,NaN,NaN
3,15614,a,NaN,NaN,0.048733,NaN,NaN,0.394737,NaN,NaN,...,NaN,NaN,NaN,NaN,b,NaN,NaN,NaN,NaN,NaN
4,8149,b,NaN,NaN,0.049924,NaN,NaN,0.368421,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,200,b,NaN,NaN,0.014728,NaN,NaN,0.368421,NaN,0.059524,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN
1996,10876,a,NaN,NaN,0.052415,NaN,NaN,0.578947,NaN,0.000000,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN
1997,4035,a,NaN,NaN,0.038770,NaN,NaN,0.368421,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,b,NaN,NaN,NaN
1998,3640,b,NaN,NaN,0.039420,NaN,NaN,0.552632,NaN,NaN,...,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN


In [26]:
test_desc = test_df.describe(include="all")
test_desc

,id,release,n_0000,n_0001,n_0002,n_0003,n_0004,n_0005,n_0006,n_0007,...,c_1368,c_1369,c_1370,c_1371,c_1372,c_1373,c_1374,c_1375,c_1376,c_1377
count,2000.000000,2000,2.000000,116.000000,1904.000000,20.000000,14.000000,1903.000000,373.000000,258.000000,...,0.0,12,109,0.0,1999,108,779,149,1,495
unique,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7,2,NaN,2,5,13,15,1,2
top,NaN,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,m,b,NaN,a,a,b,j,a,b
freq,NaN,940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4,91,NaN,1841,41,391,53,1,372
mean,9166.887500,NaN,0.384615,0.658046,0.041033,0.107341,0.295238,0.397157,0.213405,0.009690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,5322.849729,NaN,0.217571,0.208902,0.024645,0.166684,0.188043,0.084669,0.252157,0.018267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,23.000000,NaN,0.230769,0.000000,0.006714,0.000000,0.066667,0.026316,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,4502.500000,NaN,0.307692,0.452381,0.032164,0.025794,0.141667,0.342105,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,9236.000000,NaN,0.384615,0.678571,0.041585,0.063492,0.283333,0.394737,0.200000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,13802.250000,NaN,0.461538,0.857143,0.049085,0.105159,0.391667,0.421053,0.400000,0.011905,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
test_na_cols = test_desc.loc["count"][test_desc.loc["count"] == 0].index
test_na_cols

Index(['n_0008', 'n_0011', 'n_0055', 'n_0101', 'n_0111', 'o_0117', 'o_0121',
       'o_0122', 'o_0126', 'o_0149',
       ...
       'c_1288', 'c_1292', 'c_1308', 'c_1312', 'c_1332', 'c_1334', 'c_1349',
       'c_1360', 'c_1368', 'c_1371'],
      dtype='object', length=179)

Тут уже много фичей неопределено

#### Удостоверимся, что у нас одинаковые фичи там и там

In [28]:
train_df.columns.intersection(test_df.columns)

Index(['id', 'release', 'n_0000', 'n_0001', 'n_0002', 'n_0003', 'n_0004',
       'n_0005', 'n_0006', 'n_0007',
       ...
       'c_1368', 'c_1369', 'c_1370', 'c_1371', 'c_1372', 'c_1373', 'c_1374',
       'c_1375', 'c_1376', 'c_1377'],
      dtype='object', length=1379)